In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
from smodels.decomposition.theorySMS import TheorySMS
from smodels.experiment.expSMS import ExpSMS
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.experiment.defaultFinalStates import finalStates
from smodels.base.model import Model
from collections import OrderedDict
from smodels.base.physicsUnits import fb, GeV
from itertools import product
from smodels.experiment.databaseObj import Database
import copy
import re
from smodels.base.smodelsLogging import logger
import plotly.graph_objects as go

logger.setLevel('ERROR')

In [2]:
def convertAxesStr(inStr):
    entryDict = {}
    # First find the outermost enclosing brackets:
    iLeft = inStr.find('[')
    iRight = inStr.rfind(']')+1
    newStr = inStr[iLeft:iRight]
    # Remove blank spaces:
    newStr = newStr.replace(" ","")
    iel = 0
    # Loop over the string and replace entries by
    # "Xi" strings
    for x in re.split(r',\s*(?![^()]*\))', newStr):
        x = x.replace('[','').replace(']','')
        x = x.strip()
        if not x in entryDict:
            entryDict[x] = "'X%i'" %iel
            iel += 1
        xNew = entryDict[x]
        # print('x=',x,'xNew=',xNew)
        # print('before=',newStr)
        newStr = newStr.replace(x,xNew,1)
        # print('after=',newStr)
    # Add the original enclosing strings 
    # (for cases where the string ends in ]]*2)
    newStr = inStr[:iLeft]+newStr+inStr[iRight:]
    # print('final=',newStr)
    # Evaluate the string with the entries replaces
    # by Xi strings:
    axes = eval(newStr)
    
    return axes

def getAxesSMS(txAxes):
    """
    Generates a dummy SMS with nodes holding axis labels.
    """
    axesSMS = []
    if isinstance(txAxes,str):
        txAxes = [txAxes]
    for txAx in txAxes:
        axis = convertAxesStr(txAx)
        axisStr = str([[['anySM'] for ptc in br[:-1]] for br in axis])
        finalStates = [str(br[-1]) for br in axis]
        intermediateStates = [[str(ptc) for ptc in br[:-1]] for br in axis]
        axSMS = ExpSMS.from_string(axisStr,finalState=finalStates,
                                intermediateState=intermediateStates)
        axesSMS.append(axSMS)
    
    return axesSMS

def symmetricAxes(txAxes):

    if isinstance(txAxes,str):
        txAxes = [txAxes]
    for txAx in txAxes:
        axis = convertAxesStr(txAx)
        if len(axis) != 2:
            # print('Found %i branches!' %(len(axis)))
            return False
        if axis[0] != axis[1]:
            # print('Non-symmetric axis:\n %s != %s' %(axis[0],axis[1]))
            return False

    return True


### Get database

In [3]:
db = Database('official')

In [4]:
databaseSMSDict = {}
uniqueSMSDict = {}
allSMS = []
for iexp,exp in enumerate(db.expResultList):
    expID = exp.globalInfo.id
    for ids,dataset in enumerate(exp.datasets):
        dsID = dataset.dataInfo.dataId
        for itx,tx in enumerate(dataset.txnameList):
            txID = str(tx)
            tx.sortSMSMap()
            for sms,smsLabel in tx.smsMap.items():
                allSMS.append(sms)
                smsMatch = None
                for sms1 in databaseSMSDict:
                    if sms.identicalTo(sms1):
                        smsMatch = sms1
                        break
                                
                # Update dictionary
                if smsMatch is None:
                    databaseSMSDict[sms] = {iexp : {ids : {itx : smsLabel}}}
                    uniqueSMSDict[sms] = {expID : {dsID : {txID : smsLabel}}}
                else:
                    useDict = databaseSMSDict[smsMatch]
                    entry = [iexp,ids,itx]
                    for key in entry:
                        if useDict.get(key) is None:
                            useDict.update({key : {}})
                        useDict = useDict.get(key)
                    useDict[itx] = smsLabel

                    if not expID in uniqueSMSDict[smsMatch]:
                        uniqueSMSDict[smsMatch].update({expID : {}})
                    if not dsID in  uniqueSMSDict[smsMatch][expID]:
                        uniqueSMSDict[smsMatch][expID].update({dsID : {}})
                    uniqueSMSDict[smsMatch][expID][dsID] = {txID : smsLabel}
                    

AttributeError: 'Database' object has no attribute 'getExpResults'

In [ ]:
len(uniqueSMSDict),len(allSMS)

In [ ]:
len(databaseSMSDict)

In [ ]:
for sms in list(uniqueSMSDict.keys())[:3]:
    print(sms)
    anaList = list(uniqueSMSDict[sms].keys())
    txList = set([])
    for anaID in uniqueSMSDict[sms]:
        for dataID in uniqueSMSDict[sms][anaID]:
            for txID in uniqueSMSDict[sms][anaID][dataID]:
                txList.add(txID)
    print('\t %i analyses, %i txnames' %(len(anaList),len(txList)))
    print('\t Txnames = ',txList)
    print('\t Analyses = ',anaList)
    print('\n')